In [1]:
from pyspainmobility import Mobility, Zones

import folium

import pandas as pd
import seaborn as sns

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time
import os

from shapely import LineString

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

/Users/mluca/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Figure 2

In [ ]:
# download the shape files for districts and get the geodataframe 
aggregation = 'municipalities'
zones = Zones ( zones = aggregation, version=2)
zones = zones.get_zone_geodataframe()
# convert the CRS (optional)
zones = zones.to_crs('4326')
# Center of Madrid (approximate)
madrid_coords = [40.4168, -3.7038]

# Create the Folium map
m = folium.Map(location=madrid_coords, zoom_start=10, tiles='Cartodb Positron')

folium.map.CustomPane('labels').add_to(m)

def style_function(feature):
    return {
        'fillOpacity': 0,
        'color': 'black',
        'weight': 1,
    }

folium.GeoJson(
    zones,  
    style_function=style_function
).add_to(m)

# save it to an html page
m.save(f'output_map_{aggregation}.html')

# use selenium to take a screenshot of the html page
options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1000,1000") 

driver = webdriver.Chrome(options=options)

driver.get('file://' + os.path.abspath(f'output_map_{aggregation}.html'))
time.sleep(2)

driver.save_screenshot(f'map_highres_{aggregation}.png')
driver.quit()

# Figure 3

In [ ]:
# getting 1 week of mobility data. In this case, we download the data from March 10 to March 16 
mobility_data = Mobility(version=2, zones='gaus', start_date='2022-03-10', end_date='2022-03-16')
# and we extract the OD matrices 
mobility_data.get_od_data(social_agg=True)

In [ ]:
df = pd.read_parquet('Viajes_GAU_2022-03-10_2022-03-16_v2.parquet')

# filter out trips starting from Madrid's GAU 
df = df[df['id_origin']=='GAU Madrid']

# split weekends and weekdays and average the flows without considering socio-dem
wd = df[df['date'].isin(['2022-03-10','2022-03-11','2022-03-12','2022-03-13','2022-03-14'])]
we = df[df['date'].isin(['2022-03-15','2022-03-16'])]

In [ ]:
wd = wd.groupby(['date','id_origin','id_destination'])['n_trips'].sum().reset_index()
we = we.groupby(['date','id_origin','id_destination'])['n_trips'].sum().reset_index()

In [ ]:
# get the geography for each origin-destination
zones = Zones ( zones = 'gau', version=2)
zones = zones.get_zone_geodataframe()
zones = zones[['id','geometry']]
zones.to_crs('4326', inplace=True)

wd = wd.set_index('id_origin').join(zones.set_index('id'))
wd.rename(columns={'geometry':'geometry_origin'}, inplace=True)
wd.reset_index(inplace=True)
wd = wd.set_index('id_destination').join(zones.set_index('id'))
wd.rename(columns={'geometry':'geometry_destination'}, inplace=True)
wd.reset_index(inplace=True)

we = we.set_index('id_origin').join(zones.set_index('id'))
we.rename(columns={'geometry':'geometry_origin'}, inplace=True)
we.reset_index(inplace=True)
we = we.set_index('id_destination').join(zones.set_index('id'))
we.rename(columns={'geometry':'geometry_destination'}, inplace=True)
we.reset_index(inplace=True)


# Select other main cities as destinations just to make the plot more interpretable
wd = wd[wd['id_destination'].str.contains('GAU')]
we = we[we['id_destination'].str.contains('GAU')]

# we exlcude canary islands from the plot just for visualization purposes
canary_gaus = ['GAU Gran Canaria Sur','GAU Melilla','GAU Santa Cruz de Tenerife - La Laguna', 'GAU Tenerife Sur', 'GAU Valle de la Orotava', 'GAU Las Palmas de Gran Canaria', 'GAU Arrecife']

In [ ]:
wd = wd[~wd['id_destination'].isin(canary_gaus)]
we = we[~we['id_destination'].isin(canary_gaus)]

# we drop outliers just for visualization purposes based on the weekdays so to have the same scale on the plots 
wd = wd[wd['n_trips']<wd['n_trips'].describe()['75%']]
we = we[we['n_trips']<wd['n_trips'].describe()['75%']]

In [ ]:
# Get centroids
wd = wd.set_geometry('geometry_origin')
wd['centroid_origin'] = wd['geometry_origin'].centroid
wd = wd.set_geometry('geometry_destination')
wd['centroid_destination'] = wd['geometry_destination'].centroid

# Create lines for each flow
wd['line'] = wd.apply(lambda row: LineString([row['centroid_origin'], row['centroid_destination']]), axis=1)

# Normalize
vmin, vmax = wd['n_trips'].min(), wd['n_trips'].max()
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
cmap = plt.get_cmap('YlOrRd')

def trips_to_color(val):
    rgba = cmap(norm(val))
    return mcolors.to_hex(rgba)

wd['color'] = wd['n_trips'].apply(trips_to_color)

# Center in Madrid
madrid_coords = [40.4168, -3.7038]
m = folium.Map(location=madrid_coords, zoom_start=7, tiles="CartoDB positron")

folium.GeoJson(
    zones,
    style_function=lambda x: {'color': 'black', 'weight': 1, 'fillOpacity': 0}
).add_to(m)

# Plot flows as polylines
for _, row in wd.iterrows():
    folium.PolyLine(
        locations=[ [row['centroid_origin'].y, row['centroid_origin'].x],
                    [row['centroid_destination'].y, row['centroid_destination'].x] ],
        color=row['color'],
        weight=2 + 5*(row['n_trips']/wd['n_trips'].max()),  # Thicker for more trips
        opacity=1
    ).add_to(m)

# Plot centroids as circles
for point in wd['centroid_origin'].tolist() + wd['centroid_destination'].tolist():
    folium.CircleMarker(
        location=[point.y, point.x],
        radius=4,
        color='orange',
        fill=True,
        fill_opacity=0.8,
        weight=1
    ).add_to(m)

# Save to HTML
m.save("network_wd.html")


# use selenium to take a screenshot of the html page
options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1350,1350") 

driver = webdriver.Chrome(options=options)

driver.get('file://' + os.path.abspath(f'network_wd.html'))
time.sleep(2)

driver.save_screenshot(f'network_wd.png')
driver.quit()